In [2]:
import tensorflow as tf


from sionna.channel import RayleighBlockFading
from sionna.channel import GenerateTimeChannel, ApplyTimeChannel
from sionna.channel.utils import time_lag_discrete_time_channel
class TimeChannel(tf.keras.layers.Layer):
    # pylint: disable=line-too-long
    r"""TimeChannel(channel_model, bandwidth, num_time_samples, maximum_delay_spread=3e-6, l_min=None, l_max=None, normalize_channel=False, add_awgn=True, return_channel=False, dtype=tf.complex64, **kwargs)

    Generate channel responses and apply them to channel inputs in the time domain.

    This class inherits from the Keras `Layer` class and can be used as layer
    in a Keras model.

    The channel output consists of ``num_time_samples`` + ``l_max`` - ``l_min``
    time samples, as it is the result of filtering the channel input of length
    ``num_time_samples`` with the time-variant channel filter  of length
    ``l_max`` - ``l_min`` + 1. In the case of a single-input single-output link and given a sequence of channel
    inputs :math:`x_0,\cdots,x_{N_B}`, where :math:`N_B` is ``num_time_samples``, this
    layer outputs

    .. math::
        y_b = \sum_{\ell = L_{\text{min}}}^{L_{\text{max}}} x_{b-\ell} \bar{h}_{b,\ell} + w_b

    where :math:`L_{\text{min}}` corresponds ``l_min``, :math:`L_{\text{max}}` to ``l_max``, :math:`w_b` to
    the additive noise, and :math:`\bar{h}_{b,\ell}` to the
    :math:`\ell^{th}` tap of the :math:`b^{th}` channel sample.
    This layer outputs :math:`y_b` for :math:`b` ranging from :math:`L_{\text{min}}` to
    :math:`N_B + L_{\text{max}} - 1`, and :math:`x_{b}` is set to 0 for :math:`b < 0` or :math:`b \geq N_B`.
    The channel taps :math:`\bar{h}_{b,\ell}` are computed assuming a sinc filter
    is used for pulse shaping and receive filtering. Therefore, given a channel impulse response
    :math:`(a_{m}(t), \tau_{m}), 0 \leq m \leq M-1`, generated by the ``channel_model``,
    the channel taps are computed as follows:

    .. math::
        \bar{h}_{b, \ell}
        = \sum_{m=0}^{M-1} a_{m}\left(\frac{b}{W}\right)
            \text{sinc}\left( \ell - W\tau_{m} \right)

    for :math:`\ell` ranging from ``l_min`` to ``l_max``, and where :math:`W` is
    the ``bandwidth``.

    For multiple-input multiple-output (MIMO) links, the channel output is computed for each antenna of each receiver and by summing over all the antennas of all transmitters.

    Parameters
    ----------
    channel_model : :class:`~sionna.channel.ChannelModel` object
        An instance of a :class:`~sionna.channel.ChannelModel`, such as
        :class:`~sionna.channel.RayleighBlockFading` or
        :class:`~sionna.channel.tr38901.UMi`.

    bandwidth : float
        Bandwidth (:math:`W`) [Hz]

    num_time_samples : int
        Number of time samples forming the channel input (:math:`N_B`)

    maximum_delay_spread : float
        Maximum delay spread [s].
        Used to compute the default value of ``l_max`` if ``l_max`` is set to
        `None`. If a value is given for ``l_max``, this parameter is not used.
        It defaults to 3us, which was found
        to be large enough to include most significant paths with all channel
        models included in Sionna assuming a nominal delay spread of 100ns.

    l_min : int
        Smallest time-lag for the discrete complex baseband channel (:math:`L_{\text{min}}`).
        If set to `None`, defaults to the value given by :func:`time_lag_discrete_time_channel`.

    l_max : int
        Largest time-lag for the discrete complex baseband channel (:math:`L_{\text{max}}`).
        If set to `None`, it is computed from ``bandwidth`` and ``maximum_delay_spread``
        using :func:`time_lag_discrete_time_channel`. If it is not set to `None`,
        then the parameter ``maximum_delay_spread`` is not used.

    add_awgn : bool
        If set to `False`, no white Gaussian noise is added.
        Defaults to `True`.

    normalize_channel : bool
        If set to `True`, the channel is normalized over the block size
        to ensure unit average energy per time step. Defaults to `False`.

    return_channel : bool
        If set to `True`, the channel response is returned in addition to the
        channel output. Defaults to `False`.

    dtype : tf.DType
        Complex datatype to use for internal processing and output.
        Defaults to `tf.complex64`.

    Input
    -----

    (x, no) or x:
        Tuple or Tensor:

    x :  [batch size, num_tx, num_tx_ant, num_time_samples], tf.complex
        Channel inputs

    no : Scalar or Tensor, tf.float
        Scalar or tensor whose shape can be broadcast to the shape of the
        channel outputs: [batch size, num_rx, num_rx_ant, num_time_samples].
        Only required if ``add_awgn`` is set to `True`.
        The noise power ``no`` is per complex dimension. If ``no`` is a scalar,
        noise of the same variance will be added to the outputs.
        If ``no`` is a tensor, it must have a shape that can be broadcast to
        the shape of the channel outputs. This allows, e.g., adding noise of
        different variance to each example in a batch. If ``no`` has a lower
        rank than the channel outputs, then ``no`` will be broadcast to the
        shape of the channel outputs by adding dummy dimensions after the last
        axis.

    Output
    -------
    y : [batch size, num_rx, num_rx_ant, num_time_samples + l_max - l_min], tf.complex
        Channel outputs
        The channel output consists of ``num_time_samples`` + ``l_max`` - ``l_min``
        time samples, as it is the result of filtering the channel input of length
        ``num_time_samples`` with the time-variant channel filter  of length
        ``l_max`` - ``l_min`` + 1.

    h_time : [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_time_samples + l_max - l_min, l_max - l_min + 1], tf.complex
        (Optional) Channel responses. Returned only if ``return_channel``
        is set to `True`.
        For each batch example, ``num_time_samples`` + ``l_max`` - ``l_min`` time
        steps of the channel realizations are generated to filter the channel input.
    """

    def __init__(self, channel_model, bandwidth, num_time_samples,
                 maximum_delay_spread=3e-6, l_min=None, l_max=None,
                 normalize_channel=False, add_awgn=True, return_channel=False,
                 dtype=tf.complex64, **kwargs):

        super().__init__(trainable=False, dtype=dtype, **kwargs)

        # Setting l_min and l_max to default values if not given by the user
        l_min_default, l_max_default = time_lag_discrete_time_channel(bandwidth,
                                                            maximum_delay_spread)
        if l_min is None:
            l_min = l_min_default
        if l_max is None:
            l_max = l_max_default

        self._cir_sampler = channel_model
        self._bandwidth = bandwidth
        self._num_time_steps = num_time_samples
        self._l_min = l_min
        self._l_max = l_max
        self._l_tot = l_max-l_min+1
        self._normalize_channel = normalize_channel
        self._add_awgn = add_awgn
        self._return_channel = return_channel

    def build(self, input_shape): #pylint: disable=unused-argument

        self._generate_channel = GenerateTimeChannel(self._cir_sampler,
                                                     self._bandwidth,
                                                     self._num_time_steps,
                                                     self._l_min,
                                                     self._l_max,
                                                     self._normalize_channel)
        self._apply_channel = ApplyTimeChannel( self._num_time_steps,
                                                self._l_tot,
                                                self._add_awgn,
                                                tf.as_dtype(self.dtype))

    def call(self, inputs):

        if self._add_awgn:
            x, no = inputs
        else:
            x = inputs

        h_time = self._generate_channel(tf.shape(x)[0])
        if self._add_awgn:
            y = self._apply_channel([x, h_time, no])
        else:
            y = self._apply_channel([x, h_time])

        if self._return_channel:
            return y, h_time
        else:
            return y


In [3]:
# 定义一个简单的测试代码
def test_time_channel():
    batch_size = 2
    num_tx = 1
    num_tx_ant = 1
    num_rx = 1
    num_rx_ant = 1
    num_time_samples = 10
    bandwidth = 1e6

    # 模拟的channel_model，可以使用任何合适的模型实例化
    channel_model = RayleighBlockFading(num_rx,num_rx_ant,num_tx,num_tx_ant,dtype=tf.complex64)

    # 实例化 TimeChannel
    time_channel = TimeChannel(channel_model, bandwidth, num_time_samples)

    # 生成一些随机输入数据
    x = tf.complex(
        tf.random.normal([batch_size, num_tx, num_tx_ant, num_time_samples]),
        tf.random.normal([batch_size, num_tx, num_tx_ant, num_time_samples])
    )
    no = tf.constant(0.1, dtype=tf.float32)

    # 调用 TimeChannel
    y = time_channel((x, no))

    print("输入 x:")
    print(x)
    print("输出 y:")
    print(y)

# 运行测试代码
test_time_channel()

输入 x:
tf.Tensor(
[[[[-1.6753638 +0.30501944j  0.9829815 -1.004707j
    -0.43382445+0.07296482j  1.1888292 -0.36816195j
     1.6569929 -0.09330415j  0.1253348 +0.636844j
     1.7838709 +0.77345973j  1.0827578 +1.0736915j
     0.37426516+1.2467664j   0.66080546+0.3196252j ]]]


 [[[ 0.28745562-1.0514264j  -0.51575756-1.7484419j
     0.15059014-0.614426j    0.7247111 -0.4713373j
    -0.31492364-0.7146405j   0.56923485-0.1777594j
    -0.7117779 -0.7438315j   1.0640773 -0.8405494j
    -0.25746384+0.17083897j  0.17408323-0.05860179j]]]], shape=(2, 1, 1, 10), dtype=complex64)
输出 y:
tf.Tensor(
[[[[-0.01393137+3.72437626e-01j  0.05722164-7.69637234e-04j
    -0.16455391+4.74788338e-01j  0.31704393+2.28539854e-02j
     0.1984425 +5.76564819e-02j  0.11931367+2.00089335e-01j
     0.92348546-5.00796080e-01j -0.26727003+1.13903093e+00j
     0.1435872 +5.95897436e-05j -0.70640695+2.83713162e-01j
    -0.84751976+1.62441507e-01j -0.31377095-3.85421813e-01j
    -0.6261524 +5.68508580e-02j -0.6775814 -5.3

In [10]:
import torch
import torch.nn as nn

from my_code.mysionna.channel.torch_version.generate_time_channel import GenerateTimeChannel
from my_code.mysionna.channel.torch_version.apply_time_channel import ApplyTimeChannel

from my_code.mysionna.channel.torch_version.rayleigh_block_fading import RayleighBlockFading

from my_code.mysionna.channel.torch_version.utils import time_lag_discrete_time_channel
class TimeChannel(nn.Module):
    r"""TimeChannel(channel_model, bandwidth, num_time_samples, maximum_delay_spread=3e-6, l_min=None, l_max=None, normalize_channel=False, add_awgn=True, return_channel=False, dtype=torch.complex64)

    Generate channel responses and apply them to channel inputs in the time domain.

    This class inherits from the PyTorch `nn.Module` class and can be used as a layer
    in a PyTorch model.

    The channel output consists of `num_time_samples` + `l_max` - `l_min`
    time samples, as it is the result of filtering the channel input of length
    `num_time_samples` with the time-variant channel filter  of length
    `l_max` - `l_min` + 1. In the case of a single-input single-output link and given a sequence of channel
    inputs `x_0, ..., x_{N_B}`, where `N_B` is `num_time_samples`, this
    layer outputs:

    .. math::
        y_b = \sum_{\ell = L_{\text{min}}}^{L_{\text{max}}} x_{b-\ell} \bar{h}_{b,\ell} + w_b

    where `L_{\text{min}}` corresponds `l_min`, `L_{\text{max}}` to `l_max`, `w_b` to
    the additive noise, and `\bar{h}_{b,\ell}` to the
    `\ell^{th}` tap of the `b^{th}` channel sample.
    This layer outputs `y_b` for `b` ranging from `L_{\text{min}}` to
    `N_B + L_{\text{max}} - 1`, and `x_{b}` is set to 0 for `b < 0` or `b \geq N_B`.
    The channel taps `\bar{h}_{b,\ell}` are computed assuming a sinc filter
    is used for pulse shaping and receive filtering. Therefore, given a channel impulse response
    `(a_{m}(t), \tau_{m}), 0 \leq m \leq M-1`, generated by the `channel_model`,
    the channel taps are computed as follows:

    .. math::
        \bar{h}_{b, \ell}
        = \sum_{m=0}^{M-1} a_{m}\left(\frac{b}{W}\right)
            \text{sinc}\left( \ell - W\tau_{m} \right)

    for `\ell` ranging from `l_min` to `l_max`, and where `W` is
    the `bandwidth`.

    For multiple-input multiple-output (MIMO) links, the channel output is computed for each antenna of each receiver and by summing over all the antennas of all transmitters.

    Parameters
    ----------
    channel_model : object
        An instance of a ChannelModel, such as RayleighBlockFading or UMi.

    bandwidth : float
        Bandwidth (W) [Hz]

    num_time_samples : int
        Number of time samples forming the channel input (N_B)

    maximum_delay_spread : float
        Maximum delay spread [s].
        Used to compute the default value of `l_max` if `l_max` is set to `None`. If a value is given for `l_max`, this parameter is not used.
        It defaults to 3us, which was found
        to be large enough to include most significant paths with all channel
        models included in Sionna assuming a nominal delay spread of 100ns.

    l_min : int
        Smallest time-lag for the discrete complex baseband channel (L_{\text{min}}).
        If set to `None`, defaults to the value given by `time_lag_discrete_time_channel`.

    l_max : int
        Largest time-lag for the discrete complex baseband channel (L_{\text{max}}).
        If set to `None`, it is computed from `bandwidth` and `maximum_delay_spread`
        using `time_lag_discrete_time_channel`. If it is not set to `None`,
        then the parameter `maximum_delay_spread` is not used.

    add_awgn : bool
        If set to `False`, no white Gaussian noise is added.
        Defaults to `True`.

    normalize_channel : bool
        If set to `True`, the channel is normalized over the block size
        to ensure unit average energy per time step. Defaults to `False`.

    return_channel : bool
        If set to `True`, the channel response is returned in addition to the
        channel output. Defaults to `False`.

    dtype : torch.dtype
        Complex datatype to use for internal processing and output.
        Defaults to `torch.complex64`.

    Inputs
    ------
    (x, no) or x:
        Tuple or Tensor:

    x : [batch size, num_tx, num_tx_ant, num_time_samples], torch.complex64
        Channel inputs

    no : Scalar or Tensor, torch.float
        Scalar or tensor whose shape can be broadcast to the shape of the
        channel outputs: [batch size, num_rx, num_rx_ant, num_time_samples].
        Only required if `add_awgn` is set to `True`.
        The noise power `no` is per complex dimension. If `no` is a scalar,
        noise of the same variance will be added to the outputs.
        If `no` is a tensor, it must have a shape that can be broadcast to
        the shape of the channel outputs. This allows, e.g., adding noise of
        different variance to each example in a batch. If `no` has a lower
        rank than `no` will be broadcast to the shape of the channel outputs by adding dummy dimensions after the last axis.

    Outputs
    -------
    y : [batch size, num_rx, num_rx_ant, num_time_samples + l_max - l_min], torch.complex64
        Channel outputs
        The channel output consists of `num_time_samples` + `l_max` - `l_min`
        time samples, as it is the result of filtering the channel input of length
        `num_time_samples` with the time-variant channel filter of length
        `l_max` - `l_min` + 1.

    h_time : [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_time_samples + l_max - l_min, l_max - l_min + 1], torch.complex64
        (Optional) Channel responses. Returned only if `return_channel`
        is set to `True`.
        For each batch example, `num_time_samples` + `l_max` - `l_min` time
        steps of the channel realizations are generated to filter the channel input.
    """


    def __init__(self, channel_model, bandwidth, num_time_samples,
                 maximum_delay_spread=3e-6, l_min=None, l_max=None,
                 normalize_channel=False, add_awgn=True, return_channel=False,
                 dtype=torch.complex64, **kwargs) :
        super().__init__()

        # Setting l_min and l_max to default values if not given by the user
        l_min_default, l_max_default = time_lag_discrete_time_channel(bandwidth,
                                                            maximum_delay_spread)
        if l_min is None:
            l_min = l_min_default
        if l_max is None:
            l_max = l_max_default

        self._cir_sampler = channel_model
        self._bandwidth = bandwidth
        self._num_time_steps = num_time_samples
        self._l_min = l_min
        self._l_max = l_max
        self._l_tot = l_max-l_min+1
        self._normalize_channel = normalize_channel
        self._add_awgn = add_awgn
        self._return_channel = return_channel

        self._generate_channel = GenerateTimeChannel(self._cir_sampler,
                                                     self._bandwidth,
                                                     self._num_time_steps,
                                                     self._l_min,
                                                     self._l_max,
                                                     self._normalize_channel)
        self._apply_channel = ApplyTimeChannel( self._num_time_steps,
                                                self._l_tot,
                                                self._add_awgn,
                                                dtype=dtype)
        
    def forward(self, inputs):

        if self._add_awgn:
            x, no = inputs
        else:
            x = inputs
        h_time = self._generate_channel(x.shape[0])
        if self._add_awgn:
            y = self._apply_channel([x, h_time, no])
        else:
            y = self._apply_channel([x, h_time])
        
        if self._return_channel:
            return y, h_time
        else:
            return y

In [11]:
# 定义一个简单的测试代码
def test_time_channel():
    batch_size = 2
    num_tx = 1
    num_tx_ant = 1
    num_rx = 1
    num_rx_ant = 1
    num_time_samples = 10
    bandwidth = 1e6

    # 模拟的channel_model，可以使用任何合适的模型实例化
    channel_model = RayleighBlockFading(num_rx,num_rx_ant,num_tx,num_tx_ant,dtype=torch.complex64)

    # 实例化 TimeChannel
    time_channel = TimeChannel(channel_model, bandwidth, num_time_samples)

    # 生成一些随机输入数据
    x = torch.complex(
        torch.randn([batch_size, num_tx, num_tx_ant, num_time_samples]),
        torch.randn([batch_size, num_tx, num_tx_ant, num_time_samples])
    )
    no = torch.tensor(0.1, dtype=torch.float32)

    # 调用 TimeChannel
    y = time_channel((x, no))

    print("输入 x:")
    print(x)
    print(x.shape)
    print("输出 y:")
    print(y)
    print(y.shape)

# 运行测试代码
test_time_channel()

输入 x:
tensor([[[[-0.9559-0.0227j, -0.5990+0.8445j,  1.1011+0.6300j,  2.0859-0.1052j,
           -0.4886-1.1621j, -0.3858+1.2630j,  0.9367+0.8431j, -0.0325-0.5966j,
            0.5165-0.0517j, -0.2132+0.9363j]]],


        [[[ 0.2713+0.4352j,  0.0772+0.4346j, -0.5520-0.5838j,  1.0532-0.5001j,
            0.8292+0.4117j,  0.4373+0.5244j,  0.6670+0.9093j, -0.4013-0.3798j,
            0.2102-0.7558j,  1.6349+1.5615j]]]])
torch.Size([2, 1, 1, 10])
输出 y:
tensor([[[[ 0.3012-1.0980e-01j, -0.2726+4.8713e-02j,  0.2416-2.3518e-02j,
           -0.0292+1.1536e-01j, -0.0865-1.7738e-01j, -0.0549-1.3744e-01j,
           -0.1470-4.8799e-01j, -0.4562-4.7662e-01j, -0.3636+1.0698e+00j,
           -0.2905+1.3253e+00j,  1.1085-3.2719e-01j, -0.6763-4.7157e-01j,
           -0.8422+8.8590e-01j, -0.0248-5.1019e-02j, -0.1512+2.6981e-01j,
           -0.3043-4.1496e-01j,  0.0319+4.1290e-01j, -0.0491+6.1644e-02j,
            0.3861+1.3316e-02j,  0.0763-1.8835e-01j,  0.0332-7.3446e-02j,
            0.4344+8.7056e-02